In [1]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

import folium
from folium import plugins

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/sandeco/CanalSandeco/master/covid-19/covid-19-brasil.csv')
df.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-03-29,AC,Acrelândia,city,2,0.0,True,15256.0,1200013.0,13.10960,NaN
1,2020-03-29,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN
2,2020-03-29,AC,Rio Branco,city,31,0.0,True,407319.0,1200401.0,7.61074,NaN
3,2020-03-29,AC,NaN,state,34,0.0,True,881935.0,12.0,3.85516,NaN
4,2020-03-29,AL,Importados/Indefinidos,city,4,0.0,True,NaN,NaN,NaN,NaN


# IBGE latitude e longitude

In [9]:
cidades = pd.read_csv('https://raw.githubusercontent.com/sandeco/CanalSandeco/master/covid-19/cidades_brasil.csv')
cidades.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


In [18]:
cidades = cidades.set_index('codigo_ibge')

In [19]:
cidades.head()

,nome,latitude,longitude,capital,codigo_uf
codigo_ibge,,,,,
5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
5200100,Abadiânia,-16.19700,-48.7057,0,52
3100203,Abaeté,-19.15510,-45.4444,0,31
1500107,Abaetetuba,-1.72183,-48.8788,0,15


# Data Processing

In [13]:
cities = df.loc[df.place_type == 'city', :]
cities.place_type.unique()

array(['city'], dtype=object)

In [14]:
cities.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2020-03-29,AC,Acrelândia,city,2,0.0,True,15256.0,1200013.0,13.10960,NaN
1,2020-03-29,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN
2,2020-03-29,AC,Rio Branco,city,31,0.0,True,407319.0,1200401.0,7.61074,NaN
4,2020-03-29,AL,Importados/Indefinidos,city,4,0.0,True,NaN,NaN,NaN,NaN
5,2020-03-29,AL,Maceió,city,12,0.0,True,1018948.0,2704302.0,1.17769,NaN


In [15]:
len(cities)

2161

In [20]:
cities = cities.join(cidades, on = 'city_ibge_code')
cities.head()

,date,state,city,place_type,confirmed,deaths,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,nome,latitude,longitude,capital,codigo_uf
0,2020-03-29,AC,Acrelândia,city,2,0.0,True,15256.0,1200013.0,13.10960,NaN,Acrelândia,-9.82581,-66.8972,0.0,12.0
1,2020-03-29,AC,Porto Acre,city,1,0.0,True,18504.0,1200807.0,5.40424,NaN,Porto Acre,-9.58138,-67.5478,0.0,12.0
2,2020-03-29,AC,Rio Branco,city,31,0.0,True,407319.0,1200401.0,7.61074,NaN,Rio Branco,-9.97499,-67.8243,1.0,12.0
4,2020-03-29,AL,Importados/Indefinidos,city,4,0.0,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-03-29,AL,Maceió,city,12,0.0,True,1018948.0,2704302.0,1.17769,NaN,Maceió,-9.66599,-35.7350,1.0,27.0


In [25]:
geo_last = cities.loc[cities.is_last == True, ['city','latitude', 'longitude', 'state', 'confirmed', 'deaths']]
geo_last.head()


,city,latitude,longitude,state,confirmed,deaths
0,Acrelândia,-9.82581,-66.8972,AC,2,0.0
1,Porto Acre,-9.58138,-67.5478,AC,1,0.0
2,Rio Branco,-9.97499,-67.8243,AC,31,0.0
4,Importados/Indefinidos,NaN,NaN,AL,4,0.0
5,Maceió,-9.66599,-35.7350,AL,12,0.0


In [26]:
len(geo_last)

330

In [27]:
geo_last.confirmed.sum()

3868

In [29]:
len(geo_last.state.unique())

27

In [37]:
coordenadas = geo_last[['latitude', 'longitude', 'confirmed']]
coordenadas

,latitude,longitude,confirmed
0,-9.82581,-66.8972,2
1,-9.58138,-67.5478,1
2,-9.97499,-67.8243,31
4,NaN,NaN,4
5,-9.66599,-35.7350,12
...,...,...,...
707,-23.01040,-45.5593,1
708,-22.96980,-46.9974,1
709,-23.59930,-47.0220,2
1143,-30.84890,-51.8043,0


In [38]:
coordenadas = coordenadas.dropna()
coordenadas

,latitude,longitude,confirmed
0,-9.82581,-66.8972,2
1,-9.58138,-67.5478,1
2,-9.97499,-67.8243,31
5,-9.66599,-35.7350,12
6,-10.18490,-36.8376,1
...,...,...,...
706,-23.34870,-47.8461,1
707,-23.01040,-45.5593,1
708,-22.96980,-46.9974,1
709,-23.59930,-47.0220,2


In [44]:
baseMap = folium.Map(
    width = '100%', 
    height = '100%', 
    location = [-15.788497, - 47.879873],
    zoom_start = 4
)
baseMap

In [46]:
geo_last = geo_last.dropna()
geo_last

,city,latitude,longitude,state,confirmed,deaths
0,Acrelândia,-9.82581,-66.8972,AC,2,0.0
1,Porto Acre,-9.58138,-67.5478,AC,1,0.0
2,Rio Branco,-9.97499,-67.8243,AC,31,0.0
5,Maceió,-9.66599,-35.7350,AL,12,0.0
6,Porto Real do Colégio,-10.18490,-36.8376,AL,1,0.0
...,...,...,...,...,...,...
705,Taboão da Serra,-23.60190,-46.7526,SP,2,1.0
706,Tatuí,-23.34870,-47.8461,SP,1,0.0
707,Taubaté,-23.01040,-45.5593,SP,1,0.0
708,Valinhos,-22.96980,-46.9974,SP,1,0.0


In [47]:
for i in range(0 , len(geo_last)):
  folium.Circle(
      location = [ geo_last.iloc[i]['latitude'] , geo_last.iloc[i]['longitude'] ],
      color = '#00FF69',
      fill = '#00A1B3',
      tooltip = '<li><bold> CIDADE: ' + str(geo_last.iloc[i]['city']) + 
                '<li><bold> ESTADO: ' + str(geo_last.iloc[i]['state']) +
                '<li><bold> CASOS: ' + str(geo_last.iloc[i]['confirmed']) +
                '<li><bold> MORTES: ' + str(geo_last.iloc[i]['deaths']),
      radius = (geo_last.iloc[i]['confirmed'] ** 1.1) 
  ).add_to(baseMap)

baseMap

In [48]:
baseMap.save('map-cidades.html')